## Get Data

To download all match data from static dataset. Run `python lol/download_static_data.py` this will store all static data into `lol/static_matches.data`.

#### Load saved data

In [1]:
import utils

In [4]:
static_matches = utils.load_data('lol/static_matches.data')

## Build a network

In [2]:
import networkx as nx
import time
from collections import deque

In [3]:
import algos
reload(algos)

<module 'algos' from 'algos.pyc'>

In [4]:
from lol import riot_games_api
reload(riot_games_api)

<module 'lol.riot_games_api' from 'lol/riot_games_api.pyc'>

In [5]:
rg = riot_games_api.RiotGames('lol/riot_games_api.key')

#### Find all team mates for one summoner

In [9]:
# Seed node
summonerId = static_matches[0]['participantIdentities'][0]['player']['summonerId']

In [10]:
del static_matches

In [22]:
# Initialize -- Starting from seed
matches = {}  # Store data for all matches
discovered_summonerIds = {summonerId: True}  # Track discovered nodes
g = nx.Graph()

max_hop = 2  # Used as a stopping condition for Breadth First Search

bfs_queue = deque([{'summonerId': summonerId, 'hop': 0}])  # Queue for breadth first search
hop = 0

In [6]:
def load_state(num):
    matches = utils.load_data('matches_' + str(num))
    discovered_summonerIds = utils.load_data('discovered_summonerIds_' + str(num))
    g = utils.load_data('g_' + str(num))
    max_hop = utils.load_data('max_hop_' + str(num))
    bfs_queue = utils.load_data('bfs_queue_' + str(num))
    hop = utils.load_data('hop_' + str(num))
    return matches, discovered_summonerIds, g, max_hop, bfs_queue, hop

In [7]:
# Optional: Initialize from saved state
matches, discovered_summonerIds, g, max_hop, bfs_queue, hop = load_state(2)

In [8]:
def save_state(num, matches, discovered_summonerIds, g, max_hop, bfs_queue, hop):
    # Save state
    print "Saving state"
    utils.save_data(matches, 'matches_' + str(num))
    utils.save_data(discovered_summonerIds, 'discovered_summonerIds_' + str(num))
    utils.save_data(g, 'g_' + str(num))
    utils.save_data(max_hop, 'max_hop_' + str(num))
    utils.save_data(bfs_queue, 'bfs_queue_' + str(num))
    utils.save_data(hop, 'hop_' + str(num))

In [9]:
len(matches)

26985

In [10]:
len(bfs_queue)

88157

In [11]:
print "Network as %d nodes and %d edges" % (len(g.nodes()), len(g.edges()))

Network as 88197 nodes and 250532 edges


In [ ]:
winners = []
acs = []
memberIdxs = {100: [0, 1, 2, 3, 4], 200: [5, 6, 7, 8, 9]}

num_matches = len(matches.keys())
print num_matches

In [76]:
for i, matchId in enumerate(matches.keys()):
    print "\r%0.2f%% done" % (i/float(num_matches)*100),
    for team in matches[matchId]['teams']:
        teamId = team['teamId']
        num_team_members_in_graph = 0
        memberIds = []
        for memberIdx in memberIdxs[teamId]:
            try: 
                memberId = matches[matchId]['participantIdentities'][memberIdx]['player']['summonerId']
                memberIds += [memberId]
                if memberId in g:
                    num_team_members_in_graph += 1
            except:
                pass

        if num_team_members_in_graph == 5:
            winners += [team['winner']]
            g_team = g.subgraph(memberIds)
            acs += [nx.algebraic_connectivity(g_team)]
            
#     break

 26985
100.00% done


In [77]:
print len(winners)
print len(acs)

26830
26830


In [78]:
from scipy.stats import pearsonr

In [82]:
corr, pValue = pearsonr(acs, winners)

In [85]:
print "Correlation: %0.2f, p-value: %0.4f " % (corr, pValue)

Correlation: 0.00, p-value: 0.6563 
